In [8]:
from numpy.random import seed
seed(1)
import pickle
import numpy as np
from datetime import datetime
import os
from astropy.io import fits
from scipy.ndimage import median_filter
import math

import glob
from astropy.io import fits
from keras import backend as K


from matplotlib import pyplot as plt
from keras import regularizers, callbacks
from keras.utils.np_utils import to_categorical
from keras.layers import (Input, Dense, Activation, ZeroPadding1D, 
BatchNormalization, Flatten, Reshape, Conv1D, MaxPooling1D, Dropout,Add, LSTM,Embedding)
from keras.initializers import glorot_normal, glorot_uniform
from keras.optimizers import Adam
from keras.models import Model, load_model
# from desispec.interpolation import resample_flux

plt.rcParams['axes.prop_cycle'] = plt.cycler(color=plt.cm.tab10.colors)
#plt.rcParamsDefault['axes.prop_cycle']
plt.rcParams['font.size'] = 16
plt.rcParams['axes.grid'] = True
plt.rcParams['mathtext.fontset'] = 'dejavuserif'
plt.rc('grid',alpha=0.3,linestyle='--')

In [9]:
get_ipython().run_line_magic('matplotlib', 'inline')

#retrieve file names of hosts
files_host = np.sort(glob.glob('hosts/*.fits'))
#create array of fluxes
flux_host = []
for f in files_host:
    h = fits.open(f)
    fl = h['BRZ_FLUX'].data
    flux_host.append(fl)
flux_host = np.concatenate(flux_host)


#retrieve file names of SN    
files_SN=np.sort(glob.glob('sneia/*.fits'))
flux_SN = []
for f in files_SN:
    h = fits.open(f)
    fl = h['BRZ_FLUX'].data
    flux_SN.append(fl)
flux_SN = np.concatenate(flux_SN)

    
# Get wavelength array (there is just one).
h=fits.open(files_host[0])
wave=h['BRZ_WAVELENGTH'].data

#Get rid of 0 flux and NaN flux
flux_host_reg=[]
bad_host_counter=0
for i in flux_host:
    if np.sum(i)==0 or math.isnan(np.sum(i)):
        bad_host_counter=bad_host_counter+1
    else:
        flux_host_reg.append(i)
        
flux_SN_reg=[]
bad_SN_counter=0
for i in flux_SN:
    if np.sum(i)==0 or math.isnan(np.sum(i)):
        bad_SN_counter=bad_SN_counter+1
    else:
        flux_SN_reg.append(i)

In [10]:
#Median filter
medianFilteredFlux149=[]
for i in flux_host_reg:
    trial_median_0 = median_filter(i,149)
    medianFilteredFlux149.append(trial_median_0)
    
    
medianFilteredFlux149_SN=[]
for i in flux_SN_reg:
    trial_median_0 = median_filter(i,149)
    medianFilteredFlux149_SN.append(trial_median_0)

In [11]:
medianFilteredFlux999=[]
for i in flux_host_reg:
    trial_median_1 = median_filter(i,999)
    medianFilteredFlux999.append(trial_median_1)
    
medianFilteredFlux999_SN=[]
for i in flux_SN_reg:
    trial_median_1 = median_filter(i,999)
    medianFilteredFlux999_SN.append(trial_median_1)    


In [12]:
#fit 3rd degree polynomial to approximate the galactic continuum
polynomial_array=[]
for i in flux_host_reg:
    polynomial = np.polynomial.legendre.legfit(wave,i,3)
    polynomial_array.append(polynomial)
    
polynomial_array_SN=[]
for i in flux_SN_reg:
    polynomial = np.polynomial.legendre.legfit(wave,i,3)
    polynomial_array_SN.append(polynomial)

In [13]:
residual = []
for i in range(len(medianFilteredFlux149)):
    residual.append(medianFilteredFlux149[i] - np.polynomial.legendre.legval(wave, polynomial_array[i]))

residual_SN = []
for i in range(len(medianFilteredFlux149_SN)):
    residual_SN.append(medianFilteredFlux149_SN[i] - np.polynomial.legendre.legval(wave, polynomial_array_SN[i]))

In [14]:
residual=np.asarray(residual)
residual_SN=np.asarray(residual_SN)

In [15]:
maxflux = residual.max(axis=-1).reshape(-1,1)
minflux = residual.min(axis=-1).reshape(-1,1)
standarized_hosts = (residual - minflux)/(maxflux-minflux)

maxflux_SN = residual_SN.max(axis=-1).reshape(-1,1)
minflux_SN = residual_SN.min(axis=-1).reshape(-1,1)
standarized = (residual_SN - minflux_SN)/(maxflux_SN-minflux_SN)


In [16]:
def network(input_shape, learning_rate=0.0005, reg=0.0032, dropout=0.7436, seed=None):
    """ 
    Args:
    input_shape -- shape of the input spectra
    regularization_strength -- regularization factor
    dropout -- dropout rate
    seed -- seed of initializer
    Returns:
    model -- a Model() instance in Keras
    """

    X_input = Input(input_shape, name='Input_Spec')

    with K.name_scope('Conv_1'):
        X = Conv1D(filters=8, kernel_size=5, strides=1, padding='same',
                   kernel_regularizer=regularizers.l2(reg),
                   bias_initializer='zeros',
                   kernel_initializer=glorot_normal(seed))(X_input)
        X = BatchNormalization(axis=2)(X)
        X = Activation('relu')(X)
        X = MaxPooling1D(pool_size= 2)(X)

    with K.name_scope('Conv_2'):
        X = Conv1D(filters=16, kernel_size=5, strides=1, padding='same',
                   kernel_regularizer=regularizers.l2(reg),
                   bias_initializer='zeros',
                   kernel_initializer=glorot_normal(seed))(X)
        X = BatchNormalization(axis=2)(X)
        X = Activation('relu')(X)
        X = MaxPooling1D(2)(X)
    with K.name_scope('Conv_3'):
        X = Conv1D(filters=32, kernel_size=5, strides=1, padding='same',
                   kernel_regularizer=regularizers.l2(reg),
                   bias_initializer='zeros',
                   kernel_initializer=glorot_normal(seed))(X)
        X = BatchNormalization(axis=2)(X)
        X = Activation('relu')(X)
        X = MaxPooling1D(2)(X)
        
    with K.name_scope('Conv_4'):
        X = Conv1D(filters=64, kernel_size=5, strides=1, padding='same',
                   kernel_regularizer=regularizers.l2(reg),
                   bias_initializer='zeros',
                   kernel_initializer=glorot_normal(seed))(X)
        X = BatchNormalization(axis=2)(X)
        X = Activation('relu')(X)
        X = MaxPooling1D(2)(X)

        
    # FLATTEN -> FULLYCONNECTED
    with K.name_scope('Dense_Layer'):
        X = Flatten()(X)
        X = Dense(256, kernel_regularizer=regularizers.l2(reg),
                  activation='relu')(X)
        X = Dropout(rate=dropout, seed=seed)(X)
    
    with K.name_scope('Output_Layer'):
        X = Dense(1, kernel_regularizer=regularizers.l2(reg),
              activation='sigmoid',name='Output_Classes')(X)

    model = Model(inputs=X_input, outputs=X, name='SNnet')
    model.compile(optimizer=Adam(lr=learning_rate), loss='binary_crossentropy',
                  metrics=['accuracy'])
    return model

In [17]:
model2 = network((6265,1))







Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [18]:
x_train = np.concatenate([standarized,standarized_hosts]).reshape(-1,6265,1)
y_train = np.concatenate([np.zeros(standarized.shape[0]),np.ones(standarized_hosts.shape[0])])
permute = np.random.permutation(y_train.shape[0])

In [ ]:
hist = model2.fit(x_train[permute][:1996],y_train[permute][:1996],batch_size=64,epochs=50,
                  validation_split=0.1,shuffle=True)